# load the dataset MINIST

In [9]:
!pip install -q -U keras-tuner

In [17]:
from tensorflow import keras
from tensorflow.keras import layers
# Load data
(train_images, train_labels), (test_images, test_labels) = keras.datasets.mnist.load_data()
# Normalize
train_images = train_images.astype("float32") / 255
test_images  = test_images.astype("float32") / 255

# Reshape
train_images = train_images.reshape((60000, 28 * 28))
test_images  = test_images.reshape((10000, 28 * 28))


In [18]:
print(train_images.shape)
print(test_images.shape)

(60000, 784)
(10000, 784)


In [19]:
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [20]:
print(train_labels.shape)
print(test_labels.shape)

(60000, 10)
(10000, 10)


# Build the network model

In [21]:
from keras import models
from keras import layers
network = models.Sequential()
network.add(layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
network.add(layers.Dense(10, activation='softmax'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
network.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# Compile the network

In [ ]:
network.compile(optimizer='rmsprop',
loss='categorical_crossentropy',
metrics=['accuracy'])

# Training dataset : try to "fit" the model

In [ ]:
network.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8719 - loss: 0.4393
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 15s 11ms/step - accuracy: 0.9665 - loss: 0.1152
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9790 - loss: 0.0728
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9841 - loss: 0.0514
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - accuracy: 0.9892 - loss: 0.0361


# Test data set to evaluate

In [ ]:
test_loss, test_acc = network.evaluate(test_images, test_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9778 - loss: 0.0734


# Hyperparameter Tuning with Keras Tuner

In [68]:
import keras_tuner as kt
import keras

In [73]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Input(shape=(784,)))

    # Number of hidden layers
    for i in range(hp.Int("num_layers", 2, 10)):
        model.add(
            layers.Dense(
                units=hp.Int(f"units_{i}", min_value=32, max_value=512, step=32),
                activation="relu"
            )
        )
        model.add(layers.Dropout(
            hp.Float(f"dropout_{i}", 0.2, 0.5, step=0.1)
        ))

    model.add(layers.Dense(10, activation="softmax"))

    learning_rate = hp.Choice("learning_rate", [1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss="categorical_crossentropy", # Changed from sparse_categorical_crossentropy
        metrics=["accuracy"]
    )

    return model

In [69]:
build_model(kt.HyperParameters())

<Sequential name=sequential_6, built=True>

In [75]:
tuner = kt.RandomSearch(
    build_model,
    objective="val_accuracy",
    max_trials=20,
    directory="mnist_tuner",
    project_name="deep_mlp",
    overwrite=True # Added to ensure the latest model definition is used
)

In [76]:
tuner.search(
    train_images, train_labels,
    validation_split=0.1,
    epochs=20,
    batch_size=128, # Changed from kt.Choice to a fixed integer
    callbacks=[
        keras.callbacks.EarlyStopping(patience=3, monitor="accuracy") # Corrected monitor from 'accuarcy' to 'accuracy'
    ]
)

Trial 20 Complete [00h 01m 18s]
val_accuracy: 0.965499997138977

Best val_accuracy So Far: 0.9863333106040955
Total elapsed time: 00h 52m 10s
